# LabelFiltering - Classes in the list will be retained

In [ ]:
import os
from glob import glob
from tqdm import tqdm

save_in_place = True

# === CONFIG ===
folder_path = r"../../../../../../run/user/1000/gvfs/smb-share:server=10.15.1.24,share=video%202/Software department data/PROJECTS/Dataset/Mapillary_onlyroadlane/Roadlaneonlyfromsupernova/LaneData_2Classes/"

# Now we want to RETAIN only these classes
retain_classes = {0, 1, 2, 5, 13}

# Create output directory if not saving in place
if not save_in_place:
    output_dir = os.path.join(folder_path, "filtered_labels")
    os.makedirs(output_dir, exist_ok=True)

for txt_file in tqdm(glob(os.path.join(folder_path, "*.txt"))):
    with open(txt_file, "r") as f:
        lines = f.readlines()

    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue  # skip malformed lines

        cls = int(parts[0])
        if cls in retain_classes:  # Now we only KEEP these classes
            updated_lines.append(line.strip())

    if save_in_place:
        out_path = txt_file
    else:
        out_path = os.path.join(output_dir, os.path.basename(txt_file))

    if updated_lines:  # Only write if there's anything left
        with open(out_path, "w") as f:
            f.write("\n".join(updated_lines) + "\n")


# YOLO txt to JSON

In [ ]:
import os
import json
import cv2
from tqdm import tqdm
import shutil

# === CONFIG ===
input_folder = r"../../../../../../run/user/1000/gvfs/smb-share:server=10.15.1.24,share=video%202/Software department data/PROJECTS/Dataset/Mapillary_onlyroadlane/Roadlaneonlyfromsupernova/LaneData_2Classes/"  # Folder with YOLO .txt + images
output_folder = os.path.join(input_folder, "labelme_json")
os.makedirs(output_folder, exist_ok=True)

# Class ID to name mapping (reverse of your class_map)
id_to_class = {
    0: "marking--continuous--solid",
    1: "marking--discrete--other-marking",
    2: "marking--continuous--dashed",
    5: "marking--discrete--ambiguous",
    13: "marking-only--continuous--dashed",

}

# Loop through YOLO .txt files
for file in tqdm(os.listdir(input_folder)):
    if not file.endswith(".txt"):
        continue

    txt_path = os.path.join(input_folder, file)
    image_name = file.replace(".txt", ".jpg")
    image_path = os.path.join(input_folder, image_name)
    output_image_path = os.path.join(output_folder, image_name)

    if not os.path.exists(image_path):
        print(f"Image missing for {file}")
        continue

    # Read image to get width/height
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read {image_path}")
        continue
    h, w = image.shape[:2]

    # Prepare LabelMe JSON structure
    labelme_data = {
        "version": "5.3.1",  # LabelMe version
        "flags": {},
        "shapes": [],
        "imagePath": image_name,
        "imageHeight": h,
        "imageWidth": w
    }

    # Read YOLO lines
    with open(txt_path, "r") as f:
        lines = f.readlines()

    shutil.copyfile(image_path, output_image_path)

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue

        class_id = int(parts[0])
        coords = list(map(float, parts[1:]))

        label = id_to_class.get(class_id, str(class_id))

        # Case 1: Bounding box (YOLO detection format)
        if len(coords) == 4:
            x_center, y_center, bw, bh = coords
            x_center *= w
            y_center *= h
            bw *= w
            bh *= h

            x1 = x_center - bw / 2
            y1 = y_center - bh / 2
            x2 = x_center + bw / 2
            y2 = y_center + bh / 2

            shape = {
                "label": label,
                "points": [[x1, y1], [x2, y2]],
                "group_id": None,
                "shape_type": "rectangle",
                "flags": {}
            }

        # Case 2: Polygon (YOLO segmentation format)
        else:
            points = []
            for i in range(0, len(coords), 2):
                x = coords[i] * w
                y = coords[i + 1] * h
                points.append([x, y])

            shape = {
                "label": label,
                "points": points,
                "group_id": None,
                "shape_type": "polygon",
                "flags": {}
            }

        labelme_data["shapes"].append(shape)

    # Save JSON
    json_name = file.replace(".txt", ".json")
    json_path = os.path.join(output_folder, json_name)
    with open(json_path, "w") as f:
        json.dump(labelme_data, f, indent=4)

    # print(f"Converted: {file}")

print("\n✅ Conversion Complete! LabelMe JSON files saved in:", output_folder)